In [1]:
import numpy as np
import pandas as pd
import os
import re
from glob import glob

In [2]:
G19_raw_data = sorted(glob('./raw_data/G19/G*.txt'))

In [3]:
G19_raw_data_name = list(map(lambda x:x[15:],G19_raw_data))

In [4]:
i = 0
all_data = pd.DataFrame()
for file in G19_raw_data:
    data = pd.read_csv(file, sep = '\t')
    data['data_name'] = G19_raw_data_name[i]
    data.columns = data.columns.str.replace('  ','')
    data.columns = data.columns.str.replace('G19 -','G19-') #有些奇怪的間隔
    data['data_point'] = data.index
    all_data = pd.concat([data,all_data])
    all_data = all_data.drop([0]) #消除單位Deg.
    all_data.reset_index(drop = True)
    i = i + 1

C:\Users\AutoUser\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
all_data.columns

Index(['G19-PTC1', 'G19-PTC10', 'G19-PTC11', 'G19-PTC12', 'G19-PTC13',
       'G19-PTC14', 'G19-PTC15', 'G19-PTC16', 'G19-PTC17', 'G19-PTC18',
       'G19-PTC19', 'G19-PTC2', 'G19-PTC20', 'G19-PTC21', 'G19-PTC22',
       'G19-PTC23', 'G19-PTC24', 'G19-PTC25', 'G19-PTC26', 'G19-PTC27',
       'G19-PTC28', 'G19-PTC29', 'G19-PTC3', 'G19-PTC30', 'G19-PTC31',
       'G19-PTC32', 'G19-PTC33', 'G19-PTC34', 'G19-PTC35', 'G19-PTC36',
       'G19-PTC37', 'G19-PTC38', 'G19-PTC39', 'G19-PTC4', 'G19-PTC40',
       'G19-PTC41', 'G19-PTC42', 'G19-PTC43', 'G19-PTC44', 'G19-PTC45',
       'G19-PTC46', 'G19-PTC47', 'G19-PTC48', 'G19-PTC5', 'G19-PTC6',
       'G19-PTC7', 'G19-PTC8', 'G19-PTC9', 'Unnamed: 6', 'Unnamed: 8',
       'data_name', 'data_point'],
      dtype='object')

In [6]:
#drop 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8' 的columns
#rows出現 Deg的資料 欄位
for column in all_data.columns:
    if column[0] == 'U':
        all_data.drop([column],axis = 1,inplace = True)

In [7]:
all_data['data_name'] = all_data['data_name'].apply(lambda x:x[:-11]) #去掉 _Export.txt

In [8]:
all_data['data_name'].tail()

258    G19-AC(12X30)20160412-004
259    G19-AC(12X30)20160412-004
260    G19-AC(12X30)20160412-004
261    G19-AC(12X30)20160412-004
262    G19-AC(12X30)20160412-004
Name: data_name, dtype: object

In [9]:
all_data['recipes'] = all_data['data_name'].str.extract(r'(G\d+)',expand = False) #抓出所屬的Recipes
all_data['autoclave'] = all_data['data_name'].str.extract(r'(AC\(\d+x\d+\))',expand = False,flags=re.IGNORECASE) #用正則抓出 所屬的熱壓爐
all_data['date'] = all_data['data_name'].str.extract(r'(\d{7,8})',expand = False) #抓出日期
all_data['number'] = all_data['data_name'].str.extract(r'(00\d{1,3})',expand = False)#抓出所屬編號
all_data.drop(['data_name'],axis = 1,inplace = True)

In [10]:
all_data.columns = all_data.columns.str.replace('G19-','')

In [11]:
PTC_list = []
for column in all_data.columns:
    if column[0] == 'P' and column[-1] != 't':
        PTC_list.append(column)

In [12]:
all_data['PTC_count'] = len(PTC_list) - (all_data.shape[1] - all_data.count(axis = 1)) 
#每筆資料所收集溫度參數的個數

In [13]:
all_data.columns

Index(['PTC1', 'PTC10', 'PTC11', 'PTC12', 'PTC13', 'PTC14', 'PTC15', 'PTC16',
       'PTC17', 'PTC18', 'PTC19', 'PTC2', 'PTC20', 'PTC21', 'PTC22', 'PTC23',
       'PTC24', 'PTC25', 'PTC26', 'PTC27', 'PTC28', 'PTC29', 'PTC3', 'PTC30',
       'PTC31', 'PTC32', 'PTC33', 'PTC34', 'PTC35', 'PTC36', 'PTC37', 'PTC38',
       'PTC39', 'PTC4', 'PTC40', 'PTC41', 'PTC42', 'PTC43', 'PTC44', 'PTC45',
       'PTC46', 'PTC47', 'PTC48', 'PTC5', 'PTC6', 'PTC7', 'PTC8', 'PTC9',
       'data_point', 'recipes', 'autoclave', 'date', 'number', 'PTC_count'],
      dtype='object')

In [14]:
all_data['TemParameter'] = 0

In [15]:
G19_data = pd.DataFrame()

In [16]:
for PTC in PTC_list:
    data = all_data.loc[~all_data[PTC].isnull(),
             ['number', 'date', 'autoclave', 'recipes'
              ,'TemParameter', 'PTC_count','data_point',PTC]] #將各別 PTC 非0的數據挑出
    data['TemParameter'] = PTC  #建立 TemParameter 對應所屬來的PTC
    data['value'] = data[PTC]  # 賦予value值 統一
    data.drop([PTC],inplace=True,axis = 1)
    G19_data = pd.concat([G19_data,data])

In [17]:
G19_data.reset_index(drop=True, inplace=True)

In [18]:
G19_data['TemParameter'].value_counts()

PTC8     2632
PTC9     2297
PTC26    2287
PTC12    2266
PTC44    2121
PTC31    2065
PTC11    1992
PTC4     1873
PTC29    1842
PTC45    1835
PTC25    1820
PTC10    1778
PTC7     1772
PTC28    1585
PTC19    1582
PTC18    1575
PTC3     1559
PTC27    1539
PTC17    1355
PTC5     1348
PTC30    1344
PTC34    1328
PTC38    1312
PTC2     1296
PTC20    1275
PTC32    1273
PTC23    1060
PTC37    1059
PTC42    1055
PTC48    1049
PTC16    1043
PTC39    1028
PTC15    1023
PTC33    1008
PTC6      982
PTC43     816
PTC35     815
PTC22     806
PTC36     798
PTC46     797
PTC13     775
PTC24     546
PTC41     532
PTC14     516
PTC47     515
PTC1      285
PTC40     259
PTC21     256
Name: TemParameter, dtype: int64

In [19]:
G19_data.to_csv('./training_data/G19_data.csv')